# Modal Logic in Logos Theory

This notebook explores modal logic operators (□ and ◇) within the Logos semantic framework. Modal logic extends classical logic with operators for necessity and possibility, enabling reasoning about what must be true, what might be true, and relationships between different possible worlds.

We'll examine both **invalid modal arguments** (countermodels) and **valid modal principles** (theorems) to understand the subtleties of modal reasoning.

## Setup and Theory Configuration

In [1]:
# Import required modules for modal logic model checking
import sys
from model_checker.jupyter import create_build_example, build_and_check
from model_checker.theory_lib import logos

# Load modal operators along with extensional base
from model_checker.theory_lib.logos.semantic import LogosSemantics, LogosProposition, LogosModelStructure
from model_checker.theory_lib.logos.operators import LogosOperatorRegistry

# Create operator registry for modal theory
modal_registry = LogosOperatorRegistry()
modal_registry.load_subtheories(['extensional', 'modal'])

# Define the semantic theory with modal operators
modal_theory = {
    "semantics": LogosSemantics,
    "proposition": LogosProposition,
    "model": LogosModelStructure,
    "operators": modal_registry.get_operators(),
}

print(f"Loaded modal theory with {len(modal_theory['operators'].operator_dictionary)} operators")
print("Key modal operators: Box (□), Diamond (◇)")

Loaded modal theory with 13 operators
Key modal operators: Box (□), Diamond (◇)


---

## Example 1: Possibility Does Not Entail Necessity

### Background
A fundamental distinction in modal logic is between *possibility* and *necessity*. Just because something is possible doesn't mean it's necessary. This captures the intuition that contingent facts exist - things that are true but could have been otherwise.

### The Argument
- **Premise**: ◇A (A is possible)
- **Invalid Conclusion**: □A (A is necessary)
- **Why it fails**: Something can be possible in some worlds without being true in all worlds

In [ ]:
# MOD_CM_1: Test possibility does not entail necessity
MOD_CM_1_example = [
    ['\\Diamond A'],      # Premise: A is possible
    ['\\Box A'],          # Conclusion: A is necessary (invalid!)
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 2,
        'iterate': 2,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('MOD_CM_1', modal_theory, MOD_CM_1_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'MOD_CM_1',
        'Possibility to Necessity',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected!)")

### Result Interpretation
The countermodel demonstrates a scenario where A is possible (true in at least one accessible world) but not necessary (false in at least one accessible world). This captures the essence of contingency - the space between the merely possible and the necessarily true.

---

## Example 2: Possibility to Actuality

### Background
Another common modal fallacy is inferring that because something is possible, it must be actually true. This conflates what *could be* with what *is*. Modal logic carefully distinguishes between truth at the actual world and truth at possible worlds.

### The Argument
- **Premise**: ◇A (A is possible)
- **Invalid Conclusion**: A (A is actually true)
- **Why it fails**: Possibility means truth in some accessible world, not necessarily the actual world

In [ ]:
# MOD_CM_2: Test possibility does not entail actuality
MOD_CM_2_example = [
    ['\\Diamond A'],      # Premise: A is possible
    ['A'],                # Conclusion: A is true (invalid!)
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 2,
        'iterate': 1,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('MOD_CM_2', modal_theory, MOD_CM_2_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'MOD_CM_2',
        'Possibility to Actuality',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected!)")

### Result Interpretation
The countermodel shows a situation where A is possible (true in some accessible world) but false at the actual world. This illustrates that modal operators quantify over possible worlds, and possibility doesn't guarantee actuality.

---

## Example 3: The K Axiom (Distribution)

### Background
The *K axiom* is fundamental to normal modal logics. It states that necessity distributes over implication: if it's necessary that A implies B, and A is necessary, then B must be necessary too. This principle ensures that necessary truths are closed under logical consequence.

### The Argument
- **Premises**: □(A → B) (necessarily, if A then B), □A (A is necessary)
- **Valid Conclusion**: □B (B is necessary)
- **Why it works**: In every world where A holds (which is all of them), B must also hold

In [ ]:
# MOD_TH_5: Test Modal K Axiom
MOD_TH_5_example = [
    ['\\Box (A \\rightarrow B)', '\\Box A'],  # Premises
    ['\\Box B'],                                # Conclusion (valid!)
    {
        'N': 4,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 2,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (valid)
    }
]

print("Running model checker...")
model = create_build_example('MOD_TH_5', modal_theory, MOD_TH_5_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to the K Axiom!")
    model.model_structure.print_to(
        model.settings,
        'MOD_TH_5',
        'K Axiom',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: K Axiom")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nNecessity distributes over implication, ensuring logical closure.")
    print("=" * 70)

### Result Interpretation
No countermodel exists because the K axiom is a validity of normal modal logic. This axiom is essential for ensuring that necessary truths behave coherently with logical implication.

---

## Example 4: Modal Duality

### Background
*Modal duality* expresses the fundamental relationship between necessity and possibility through negation. Saying something is necessary is equivalent to saying its negation is not possible. This duality is analogous to the relationship between universal and existential quantifiers in predicate logic.

### The Argument
- **Premise**: □A (A is necessary)
- **Valid Conclusion**: ¬◇¬A (it's not possible that not-A)
- **Why it works**: If A is true in all worlds, then ¬A cannot be true in any world

In [ ]:
# MOD_TH_3: Test Modal Duality (Box to Diamond)
MOD_TH_3_example = [
    ['\\Box A'],                     # Premise: A is necessary
    ['\\neg \\Diamond \\neg A'],    # Conclusion: Not possible that not-A
    {
        'N': 4,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 2,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (valid)
    }
]

print("Running model checker...")
model = create_build_example('MOD_TH_3', modal_theory, MOD_TH_3_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to Modal Duality!")
    model.model_structure.print_to(
        model.settings,
        'MOD_TH_3',
        'Modal Duality',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Modal Duality")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nNecessity and possibility are interdefinable through negation.")
    print("=" * 70)

### Result Interpretation
The absence of a countermodel confirms modal duality as a theorem. This principle shows that □ and ◇ are not independent - each can be defined in terms of the other using negation, just as ∀ and ∃ are related in predicate logic.

---

## Summary

This notebook has explored fundamental aspects of modal logic:

### Invalid Inferences (Countermodels)
- **Possibility to Necessity**: ◇A does not entail □A
- **Possibility to Actuality**: ◇A does not entail A

### Valid Principles (Theorems)
- **K Axiom**: Necessity distributes over implication
- **Modal Duality**: □A entails ¬◇¬A

These examples illustrate how modal logic extends classical logic to reason about necessity, possibility, and alternative scenarios. The model-checking approach reveals the structure of possible worlds that underlies modal reasoning.